In [1]:
from transformers import MBartForConditionalGeneration, AutoTokenizer

In [2]:
from huggingface_hub import notebook_login
from transformers import MBart50Tokenizer
import tqdm

In [3]:
# checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
checkpoint = "/mnt/disk/yrajcoomar/kreol-benchmark/checkpoint/checkpoint-270000"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
import torch

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device='cpu'

In [7]:
# tokenizer = MBart50Tokenizer.from_pretrained("/mnt/disk/yrajcoomar/kreol-benchmark/pipelines/tok",max_len=256)
model = MBartForConditionalGeneration.from_pretrained(checkpoint)
model = model.to(device)

In [19]:
610,880,512

(610, 880, 512)

'en_XX'

In [34]:
# example_english_phrase = "I will not be following you wherever you go, make a decision please."
example_creole_phrase = 'Mo pa pou swiv zot kot zot ale, fer enn desizion silvouple.'

tokenizer.tgt_lang ='en_XX'
tokenizer.src_lang= 'cr_CR'

inputs = tokenizer(example_english_phrase, return_tensors="pt").to(device)

generated_tokens = model.generate(**inputs)

tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


['Mo pa pou swiv zot kot zot ale, fer enn desizion silvouple.']

In [35]:
generated_tokens

tensor([[     2, 250004,   2501,    249,  20618,     91,   2452,    334,   1602,
             18,   1975,   1602,     18,    662,      4,   1592,   8201,      8,
           4900,   1830,   5795,  24375,   8705,      5,      2]])

In [37]:
tokenizer.decode(generated_tokens[0][2:4])

'Mo pa'

In [33]:
inputs['input_ids'][0]

tensor([250004,     87,   1221,    959,    186,  25632,    398,   7440,    814,
           398,    738,      4,   3249,     10,  51957,  22936,      5,      2])

In [21]:
generated_tokens

tensor([[     2, 250054,   2501,    249,  20618,     91,   2452,    334,  80059,
           1975,     47,    662,      4,   1592,   8201,      8,   4900,   1830,
           5795,  24375,   8705,      5,      2]])

In [22]:
tokenizer("Mo nepli compran ki p arriver ici man.")['input_ids']

[250004, 2501, 108, 26026, 19004, 19, 200, 915, 101638, 25465, 332, 5, 2]

In [23]:
tokenizer.decode([3, 2501, 108, 26026, 19004, 19, 200, 915, 101638, 25465, 332, 5, 2])

'<unk> Mo nepli compran ki p arriver ici man.</s>'

In [38]:
import evaluate

bleu = evaluate.load("bleu")
chrf = evaluate.load("chrf")

In [39]:
import pandas as pd
val = pd.read_json('/mnt/disk/yrajcoomar/kreol-benchmark/data/lang_data/en-cr/en-cr_dev.jsonl',lines=True)

In [40]:
val_inputs = list(val['input'])
val_labels = list(val['target'])

In [41]:
input_tokens = tokenizer(val_inputs,max_length=128, truncation=True, padding="max_length",return_tensors='pt').to(device)

In [42]:
output_tokens = model.generate(**input_tokens)

In [43]:
output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)

In [44]:
chrf.compute(predictions=output,references=val_labels) # bidirectional

{'score': 44.51428623272526, 'char_order': 6, 'word_order': 0, 'beta': 2}

In [30]:
chrf.compute(predictions=output,references=val_labels)

{'score': 46.111429700032176, 'char_order': 6, 'word_order': 0, 'beta': 2}

In [32]:
chrf.compute(predictions=output,references=val_labels)

{'score': 44.25118855197231, 'char_order': 6, 'word_order': 0, 'beta': 2}

In [18]:
chrf.compute(predictions=output,references=val_labels)

{'score': 44.88389045784656, 'char_order': 6, 'word_order': 0, 'beta': 2}

In [45]:
bleu.compute(predictions=output,references=val_labels) #bi

{'bleu': 0.22710547384388105,
 'precisions': [0.5968992248062015,
  0.31207886199151486,
  0.18339100346020762,
  0.1127583749109052],
 'brevity_penalty': 0.911600492464646,
 'length_ratio': 0.9152870350462267,
 'translation_length': 8514,
 'reference_length': 9302}

In [31]:
bleu.compute(predictions=output,references=val_labels)

{'bleu': 0.232351109380634,
 'precisions': [0.5994308480364258,
  0.3108026554013277,
  0.18188824662813102,
  0.10883886906395827],
 'brevity_penalty': 0.9428478906353579,
 'length_ratio': 0.9444205547194152,
 'translation_length': 8785,
 'reference_length': 9302}

: 

In [33]:
bleu.compute(predictions=output,references=val_labels)

{'bleu': 0.21677202646600077,
 'precisions': [0.5791284403669725,
  0.29708029197080293,
  0.16904145077720206,
  0.0991552416562803],
 'brevity_penalty': 0.9354354658248155,
 'length_ratio': 0.9374328101483552,
 'translation_length': 8720,
 'reference_length': 9302}

In [17]:
bleu.compute(predictions=output,references=val_labels)

{'bleu': 0.22213435249592578,
 'precisions': [0.5861203059011528,
  0.3022636484687084,
  0.17265816260791136,
  0.1019003029468466],
 'brevity_penalty': 0.940117001603055,
 'length_ratio': 0.9418404644162546,
 'translation_length': 8761,
 'reference_length': 9302}

In [14]:
bleu.compute(predictions=output,references=val_labels)

{'bleu': 0.2034403402528741,
 'precisions': [0.5751253059797179,
  0.28382225522960763,
  0.16136693495184062,
  0.09110169491525423],
 'brevity_penalty': 0.9191779356555749,
 'length_ratio': 0.9222747796172867,
 'translation_length': 8579,
 'reference_length': 9302}

In [20]:
import numpy as np

In [21]:
r = list(np.random.randint(0,500,10))

In [22]:
for i in r:
    print(f"Input: {val_inputs[i]}")
    print(f"Prediction: {output[i]}")
    print(f"Label: {val_labels[i]}")
    print('------------')

Input: The disciples had forgotten to bring any bread when they crossed the lake.
Prediction: Bann disip la ti bliye amenn dipin kan zot ti travers lak.
Label: Bann disip ti finn bliye amenn dipin avek zot kan zot ariv lot kote lak.
------------
Input: Some day you will see that horrible hhingh in the holy place, just as the prophet Daniel said.
Prediction: Enn zour zot pou trouv sa move nouvel dan sa plas la, parey kouma profet Daniel ti dir.
Label: Alor ler zot trouv bann sakrilez abominab parey kouma Profet Daniel ti anonse.
------------
Input: In a democracy, policy makers and people in positions of responsibility must be constantly accountable to citizens.
Prediction: Dan enn demokrasi, bann fer politik, bann dimunn dan pozisyon de responsabilite, bizin constantly kontribye a bann sitwayin.
Label: Dan enn demokrasi, bann desider e dimounn dan pozision responsabilite bizin touletan rann kont bann sitwayin.
------------
Input: John's followers took his body and buried it.
Prediction

In [20]:
for i in r:
    print(f"Input: {val_inputs[i]}")
    print(f"Prediction: {output[i]}")
    print(f"Label: {val_labels[i]}")
    print('------------')

Input: While the Pharisees were still there, Jesus asked them, wWhat do you think about the Messiah?
Prediction: Pandan ki bann Farizien ti ankor laba, Zezi dir zot, ki zot panse lor Lemesi.
Label: Ler bann Farizien ti finn rasanble, Zezi dimann zot ki zot panse lor Lemesi?
------------
Input: When they got there, he told them, sit here while I go over there and pray.
Prediction: Ler zot ariv laba, li dir zot, asiz laba e mo fer priye.
Label: Kan zot ariv laba, li dir bann-la, res la, mwa mo pe al inpe pli lwin pou priye.
------------
Input: Mary Magdalene, Mary the mother of James and Joseph, and the mother of James and John were some of these women.
Prediction: Marie Magdalenn, Marie, mama Zak ek Josef, mama Zak ek Zan ti parmi sa bann fam la.
Label: Parmi zot, ti ena Marie Magdalenn, Marie, mama Zak ek Zozef e mama bann garson Zebede.
------------
Input: But I tell you that Solomon with all his wealth wasn't as well clothed as the flowers.
Prediction: Me mo dir zot, salme Salomon ar

In [12]:
for i in range(10):
    print(f"Input: {val_inputs[i]}")
    print(f"Prediction: {output[i]}")
    print(f"Label: {val_labels[i]}")
    print('------------')


Input: I did not come to do away with them, but to give them their full meaning.
Prediction: Mo pa ti vini avek zot, me zot ti fer zot per zot.
Label: Mo pa finn vini pou aboli me pou donn zot zot vre sinifikasion.
------------
Input: The fact is, at the time, you had to pay the teacher in order to go to school.
Prediction: Letan zot al lekol, zot ti fer nwaye dan lekol.
Label: Anverite sa lepok la pou al lekol ti ena enn fiz pou pey profeser.
------------
Input: Angina can be described as a discomfort, heaviness, pressure, aching, burning.
Prediction: Antrefwa “mili” enn antropolog, dibwa, par lafors gravite.
Label: Nou capav dekrir anzinn couma enn sensasion inkonfortab, lourder, presion.
------------
Input: The boy said he would, but he didn't go.
Prediction: Bann garson la ti pou al plis, me li pa ti ale.
Label: Garson-la reponn wi papa, li pou ale me li pa ale.
------------
Input: Was it God in heaven or merely some human being?
Prediction: Eski u ti dan enn lot ka?
Label: Eski sa

In [33]:
val_inputs[0]

'I did not come to do away with them, but to give them their full meaning.'

In [34]:
val_labels[0]

'Mo pa finn vini pou aboli me pou donn zot zot vre sinifikasion.'

In [32]:
tokenizer.batch_decode(output, skip_special_tokens=True)


['Mo pa vinn zwenn zot, me zot ti pe dimann zot.']

In [31]:
output

tensor([[    0, 20004,   890,   101,   230,  1196,    81, 19941,   190,    81,
            32,    68,  1104,    81, 19940,     2]])

In [5]:
val

,input,target
0,"I did not come to do away with them, but to gi...",Mo pa finn vini pou aboli me pou donn zot zot ...
1,"The fact is, at the time, you had to pay the t...",Anverite sa lepok la pou al lekol ti ena enn f...
2,"Angina can be described as a discomfort, heavi...",Nou capav dekrir anzinn couma enn sensasion in...
3,"The boy said he would, but he didn't go.","Garson-la reponn wi papa, li pou ale me li pa ..."
4,Was it God in heaven or merely some human being?,Eski sa ti sorti depi dan lesiel ouswa dimoun ...
...,...,...
495,"The angel answered, the Holy Spirit will come ...","Anz la reponn, Lespri Sin pou vinn lor twa, e ..."
496,"At its end, it had a whole lot of Flame-Trees ...","Dan so finision, de kote sime, ti ena enn ta p..."
497,"The king will answer, whenever you did it for ...","Lerla lerwa reponn zot, sak fwa ki zot finn fe..."
498,You Pharisees and teachers of the Law of Moses...,"Maler lor zot profeser lalwa Moiz ek Farizien,..."


In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}